### Installation

In terminal:

```
conda create -n starfish python=3.7  
conda activate starfish  
pip install starfish[napari]



In [ ]:
from starfish import data

import tifffile as tif
from glob import glob
import os
import pandas as pd

import csv
import os
import numpy as np
import shutil
import skimage.io
import tempfile
import time
import sys
import numpy as np
import copy
from scipy import spatial

from slicedimage import ImageFormat
from starfish.experiment.builder import format_structured_dataset

from starfish import Experiment
from starfish.types import Axes

# Decode spots with SimpleLookupDecoder
from starfish.spots import DecodeSpots
from starfish.spots import FindSpots


### Set original images dir:

In [ ]:
path = '/YOUR/PATH/HERE'

In [ ]:
org_images_dir = os.path.join(path, 'ORG_IMS')

In [ ]:
org_ims_paths = glob(os.path.join(org_images_dir,'*','*.tif'))

In [ ]:
# Those images are used for accuracy analysis
org_simul_ims_paths = [p for p in org_ims_paths 
                       if "embryos_FISH" not in p
                      and "3000spots" not in p
                      and os.path.exists(f'{p[:-4]}.loc')]

## Those images are used for execution time analysis:
org_embryo_ims_paths = [p for p in org_ims_paths if "embryos_FISH" in p]

### Define/create analysis directories:

In [ ]:
sf_dir = os.path.join(path, 'starfish')
os.makedirs(sf_dir, exist_ok=True)

In [ ]:
sf_prep_ims_dir = os.path.join(sf_dir, 'prep_spacext_ims')
os.makedirs(sf_prep_ims_dir, exist_ok=True)

In [ ]:
spacext_ims_dir = os.path.join(sf_dir, 'spacext_ims')
os.makedirs(spacext_ims_dir, exist_ok=True)

In [ ]:
results_path_simul = os.path.join(sf_dir, 'results')
os.makedirs(results_path_simul, exist_ok=True)

results_path_embryos = os.path.join(sf_dir, 'embryos_results')
os.makedirs(results_path_embryos, exist_ok=True)

In [ ]:
# For comparison of similar number of points as RSFISH found for each embryo, csv of data from RSFISH
# Assuming that more points take longer to analyse

df_RSFISH_results_path = 'PATH_TO/RSFISH_embryos_npoints_and_times.csv'

In [ ]:
results_embryos_df_path = 'PATH/TO/DESIRED/LOCATION/OF/EMBRYOS/RESULTS.csv'

# Convert Image Data to SpaceTx

#### Create spaceTX formated for all images:

In [ ]:
for im_path in org_ims_paths: 

    im = tif.imread(im_path)
    
    # columns: r, ch, zplane
    fovs = [
        [
            (0, 0, i) for i in range(im.shape[0])
        ],
    ]

    im_dir = os.path.basename(os.path.dirname(im_path))
    
    # Time analysis done on the embryo dir:
    if im_dir=='embryos_FISH':
        zc_max = 358.0
    # accuracy analysis done on simulated data
    else:
        str_xy = im_dir.split('Sigxy ')[1].split(' SigZ')[0]
        str_xy = 1.0+float('0.'+str_xy.split('pt')[1]) if 'pt' in str_xy else int(str_xy)
        str_z = int(im_dir.split('SigZ ')[1])
        
        zc_max = 256.0*(str_z/str_xy)
    
    coordinates_of_fovs = [
        {
            'xc_min': 0.0,
            'xc_max': 256.0,
            'yc_min': 0.0,
            'yc_max': 256.0,
            'zc_min': 0.0,
            'zc_max': zc_max,
        },
    ]

    # create example image tiles that adhere to the structured data schema
    primary_dir = os.path.join(sf_prep_ims_dir, os.path.basename(os.path.dirname(im_path)), os.path.basename(im_path)[:-4], "primary_dir")
    os.makedirs(primary_dir, exist_ok=True)

    for fov_id, fov in enumerate(fovs):
        for round_label, ch_label, zplane_label in fov:
            primary_path = os.path.join(
                primary_dir, f"primary-f{fov_id}-r{round_label}-c{ch_label}-z{zplane_label}.tiff")
            skimage.io.imsave(primary_path, im[zplane_label])

    # write coordinates file for primary:
    with open(os.path.join(primary_dir, "coordinates.csv"), "w") as fh:
        csv_writer = csv.DictWriter(
            fh,
            [
                'fov', 'round', 'ch', 'zplane',
                'xc_min', 'yc_min', 'zc_min', 'xc_max', 'yc_max', 'zc_max',
            ]
        )
        csv_writer.writeheader()
        for fov_id, (fov_info, coordinate_of_fov) in enumerate(zip(fovs, coordinates_of_fovs)):
            for round_label, ch_label, zplane_label in fov:
                tile_coordinates = coordinate_of_fov.copy()
                tile_coordinates.update({
                    'fov': fov_id,
                    'round': round_label,
                    'ch': ch_label,
                    'zplane': zplane_label,
                })
                csv_writer.writerow(tile_coordinates)
                
                
                
    primary_out = os.path.join(spacext_ims_dir, os.path.basename(os.path.dirname(im_path)), os.path.basename(im_path)[:-4], "primary")
    os.makedirs(primary_out, exist_ok=True)

    format_structured_dataset(
        primary_dir,
        os.path.join(primary_dir, "coordinates.csv"),
        primary_out,
        ImageFormat.TIFF,
    )

# Finding Spots

### Define spot finding function
We use blob_detector, but stardist has many options

In [ ]:
def find_spots(imgs, min_sigma, max_sigma, num_sigma, threshold):

    p = FindSpots.BlobDetector(
        min_sigma=min_sigma,
        max_sigma=max_sigma,
        num_sigma=num_sigma,
        threshold=threshold,
        measurement_type='mean', 
        exclude_border=False
    )
    
    intensities = p.run(image_stack=imgs, n_processes=40)
    return intensities

# Simulated Data

## Quick grid search
on the parameters (sigma and threshold) to try to optimize blob detection on the simulated images

In [ ]:
## Load image:

## Some of those more specific values were chosen after the first run.
sigs = [2,4,5]
thrs = [0.001, 0.002, 0.0001, 0.000145, 0.0000404, 0.0000412, 0.000042, 0.0000428, 0.000043, 0.0000439, 0.000044, 0.000045, 0.000046, 
        0.000047, 0.000048, 0.0000483, 0.000049, 0.00005, 0.000065125, 0.000074, 0.000075, 0.000076, 0.000077, 
        0.000078, 0.000079, 0.00008, 0.0000805, 0.00009, 0.000095]

for i,im_path in enumerate(org_simul_ims_paths):

    im_dir = os.path.basename(os.path.dirname(im_path))
    im_name = os.path.basename(im_path[:-4])
    
    # load image:
    experiment = Experiment.from_json(os.path.join(
    spacext_ims_dir, im_dir, im_name, 'primary', 'experiment.json'))

    imgs = experiment.fov().get_image('primary')

    # Find spots:

    for sig in sigs:


        for thr in thrs:

            print(im_path, i, sig, thr)

            start = time.time()

            spotss = find_spots(imgs, 0, sig, sig, thr)

            gt_n_spots = int(im_name.split('Poiss_')[1].split('spots')[0])

            if abs( n_spots - spotss.count_total_spots() ) == 0:
                flag =True

            diff_nspots = abs(gt_n_spots - spotss.count_total_spots())
            
            print(f'gt_nspots - nspots = {diff_nspots}')

            if diff_nspots < 5:

                # Decode:
                decoder = DecodeSpots.SimpleLookupDecoder(codebook=experiment.codebook)
                decoded_intensities = decoder.run(spots=spotss)

                exe_time = time.time() - start
                exe_time = f'{exe_time:.3g}'

                filename = f'{im_dir}__{im_name}__sig{sig}_thr{thr}__exetime{exe_time}.csv'

                decoded_intensities.to_dataframe("results")[["x","y","z"]].reset_index(drop=True).to_csv(
                os.path.join(results_path_simul,filename))

### Analyse spot detection accuracy:

In [ ]:
## Leo's code:

# This function compares two arrays:
# Unmod = ground truth array
# More_than = detections from one of the programs

#Function checks if points in More_than are close/match points in GT array (under certain distance)

#Returns: 
# # of undetected ground truth points
# # spurious detections
# and average distance between detection and associated points

def profile_detections(unmod, more_than):

    min_dist = 2

    distance_arr = []

    removedItems = True
    euc_dist = 0

    while (removedItems and len(more_than) != 0 and len(unmod) != 0 ):
        #print("loop")

        minDist = 10000
        minIndexUnmod = -1
        minIndexMore_Than = -1
        counter = 0
        kd_copy = copy.deepcopy(more_than)
        kdtree = spatial.KDTree(kd_copy)

        for item in unmod:
            distance,index = kdtree.query(item) # a new KD tree is made
            if ( distance < minDist ):
                minDist = distance
                minIndexUnmod = counter
                minIndexMore_Than = index
                #print(minDist, counter, item)
            counter = counter + 1

        if ( minDist < min_dist): # if less than min dist
            more_than = np.delete(more_than, minIndexMore_Than, axis = 0 ) # delete mod ind
            unmod = np.delete(unmod,minIndexUnmod, axis = 0) #delete unmod ind
            #print(len(more_than),distance) # sanity checkd
            removedItems = True
            distance_arr.append(minDist) # if we want to extrat stat ig

        else:
            removedItems = False
    if (len(distance_arr) >0):
        euc_dist = np.mean(np.asarray(distance_arr))
        
    return(len(unmod), len(more_than), euc_dist)

In [ ]:
all_results = glob(os.path.join(results_path_simul,'*.csv'))

In [ ]:
diff_results_list = []
best_results_files = []
    
for i,im_path in enumerate(org_simul_ims_paths):
    
    gt_dir = os.path.basename(os.path.dirname(im_path))
    gt_filename = os.path.basename(im_path[:-4])

    gt_path = f'{im_path[:-4]}.loc')

    df = pd.read_csv(gt_path, sep = "\s+", header=None)
    gt_spots = df.to_numpy()[:,:-1] - np.array([0.5,0.5,1])

    im_results = [r for r in all_results if gt_dir in r and gt_filename in r]

    im_final_diff_all = []

    for result_path in im_results:

        detected_spots = pd.read_csv(result_path, index_col=0)[["y","x","z"]].to_numpy()

        diff_results = list(profile_detections(gt_spots, detected_spots))

        n_spots = int(gt_path.split("spots")[0].split("_")[-1])
        
        ## make diff_results in the format of - "n_spots, FN, FP, distance"
        diff_results.insert(0, n_spots)
        diff_results = ",".join([str(d_r) for d_r in diff_results])

        im_final_diff_all.append(diff_results)

    ## Score of each result file is FN+FP (with best score is the lowest score
    scores = [int(r.split(',')[1]) + int(r.split(',')[2]) for r in im_final_diff_all]
    l_min = min(scores)
    idx = scores.index(l_min)
    best_diff = im_final_diff_all[idx]
    best_result_file = im_results[idx]

    diff_results_list.append(best_diff)
    best_results_files.append(best_result_file)


#### Save best result files:

In [ ]:
with open(os.path.join(sf_dir, 'results_nspots_missed_overdetected_distance.txt'),'w') as f:
    f.write("\n".join(diff_results_list))
    
with open(os.path.join(sf_dir,'best_result_files.txt'),'w') as f:
    f.write("\n".join(best_results_files))

## Embryos
Those images were analysed for execution time analysis

In [ ]:
# For comparison of similar number of points as RSFISH found for each embryo, load the data from RSFISH
# Assuming that more points take longer to analyse

df_rs = pd.read_csv('PATH_TO/RSFISH_embryos_npoints_and_times.csv')

In [ ]:
names = df_rs.name.values

In [ ]:
## check that all embryos exist in org dir:
embryos_names = [os.path.basename(p) for p in org_embryo_ims_paths]

## Should output nothing!
set(names) - set(embryos_names)

In [ ]:
## Take only embryos that were analysed for the rest of the tools
embryos_paths = [os.path.join(os.path.dirname(embryos_paths[0]), n) for n in names]

In [ ]:
sigs = [2]
thrs = [0.009, 0.0105, 0.011, 0.0125, 0.012, 0.013, 0.014, 0.016, 0.016, 0.02, 0.25]

for i,im_path in enumerate(embryos_paths):

    im_dir = os.path.basename(os.path.dirname(im_path))
    im_name = os.path.basename(im_path[:-4])
    
    # load image:
    experiment = Experiment.from_json(os.path.join(
    spacext_ims_dir, im_dir, im_name, 'primary', 'experiment.json'))

    imgs = experiment.fov().get_image('primary')

    # Find spots:

    for sig in sigs:

        for thr in thrs:

            start = time.time()
            
            spotss = find_spots(imgs, 0, sig, sig, thr)

            decoder = DecodeSpots.SimpleLookupDecoder(codebook=experiment.codebook)
            decoded_intensities = decoder.run(spots=spotss)

            exe_time = time.time() - start
            exe_time = f'{exe_time:.3g}'
            
            nspots = spotss.count_total_spots()
            gt_nspots = int(df_rs.at[i,"n_spots"])
            print(f'found {nspots}, GT {gt_nspots}')

            filename = f'{im_dir}__{im_name}__sig{sig}_thr{thr}__exetime{exe_time}_nspots{nspots}_RSnspots{gt_nspots}.csv'

            decoded_intensities.to_dataframe("results")[["x","y","z"]].reset_index(drop=True).to_csv(
            os.path.join(results_path_embryos,filename))

#### Compare number of spots to rs_fish

In [ ]:
all_results = glob(os.path.join(results_path_embryos,"*"))

In [ ]:
## Get all result files for each embryo

embryos_results = []

for n in names:
    
    embryo_result = [r for r in all_results if n[:-4] in r]
    
    embryos_results.append(embryo_result)

In [ ]:
best_files = []

for i in df_rs.index:
    nspots_rs = df_rs.at[i,"n_spots"]
    
    mini = 10000
    for r in embryos_results[i]:
        
        nspots = pd.read_csv(r).shape[0]
        diff = abs(nspots_rs-nspots)
        
        if diff<mini:
            best_file = r
            mini = diff
            
    best_files.append(best_file)

In [ ]:
times = [float(r.split('exetime')[1].split("_")[0]) for r in best_files]
names = [r.split("embryos_FISH__")[1].split("__")[0] for r in best_files]
n_spots = [pd.read_csv(r).shape[0] for r in best_files]

In [ ]:
df = pd.DataFrame({"time": times, 
                      "name": names,
                      "n_spots": n_spots,
                      "method": "starFISH"})

In [ ]:
df.to_csv(results_embryos_df_path, index=False)